<a href="https://colab.research.google.com/github/lwvc/FallDown/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# 資料預處理 - Falldown

In [ ]:
csv_path = "/content/drive/MyDrive/訓練/data/original_data/skeleton.csv"
df = pd.read_csv(csv_path)

df['video'] = df['image'].str.extract(r'(F\d{2})')[0] + df['image'].apply(lambda x: '_flip' if '_flip' in x else '')

df['frame'] = df['image'].str.extract(r'_(\d{3})')[0].astype(int)

print("群組:", df['video'].unique())
# print(df.head(10))

群組: ['F01' 'F02' 'F03' 'F04' 'F05' 'F06' 'F07' 'F08' 'F09' 'F10' 'F11' 'F12'
 'F15' 'F16' 'F17' 'F18' 'F19' 'F20' 'F21' 'F22' 'F23' 'F24' 'F25' 'F26'
 'F27' 'F28' 'F29' 'F30' 'F01_flip' 'F02_flip' 'F03_flip' 'F04_flip'
 'F05_flip' 'F06_flip' 'F07_flip' 'F08_flip' 'F09_flip' 'F10_flip'
 'F11_flip' 'F12_flip' 'F15_flip' 'F16_flip' 'F17_flip' 'F18_flip'
 'F19_flip' 'F20_flip' 'F21_flip' 'F22_flip' 'F23_flip' 'F24_flip'
 'F25_flip' 'F26_flip' 'F27_flip' 'F28_flip' 'F29_flip' 'F30_flip']


## 線性插值

In [ ]:
def interpolate(group, threshold=0.6):
  group_copy = group.copy()
  numeric_cols = group_copy.select_dtypes(include=['number']).columns
  for col in numeric_cols:
    missing_ratio = group_copy[col].isna().sum() / len(group_copy)
    if missing_ratio <= threshold: #缺失值少於1/3
      group_copy[col] = group_copy[col].interpolate(method="linear", limit_direction="both")
    else:
      print(f"{group_copy['video'].iloc[0]}的{col}欄位，缺失率達{missing_ratio:.1%}，不補值")
  return group_copy

interpolated = (df.groupby('video').apply(lambda g: interpolate(g, threshold=3/5)).reset_index(drop=True))
# interpolated.to_csv('/content/skeleton_interpolated.csv', index=False)

F01的right_elbow_x欄位，缺失率達64.4%，不補值
F01的right_elbow_y欄位，缺失率達64.4%，不補值
F01的right_knee_x欄位，缺失率達64.4%，不補值
F01的right_knee_y欄位，缺失率達64.4%，不補值
F01的right_ankle_x欄位，缺失率達66.7%，不補值
F01的right_ankle_y欄位，缺失率達66.7%，不補值
F01_flip的left_ankle_x欄位，缺失率達64.4%，不補值
F01_flip的left_ankle_y欄位，缺失率達64.4%，不補值
F02的right_elbow_x欄位，缺失率達91.1%，不補值
F02的right_elbow_y欄位，缺失率達91.1%，不補值
F02的right_wrist_x欄位，缺失率達95.6%，不補值
F02的right_wrist_y欄位，缺失率達95.6%，不補值
F02的right_knee_x欄位，缺失率達71.1%，不補值
F02的right_knee_y欄位，缺失率達71.1%，不補值
F02_flip的left_elbow_x欄位，缺失率達95.6%，不補值
F02_flip的left_elbow_y欄位，缺失率達95.6%，不補值
F02_flip的left_wrist_x欄位，缺失率達91.1%，不補值
F02_flip的left_wrist_y欄位，缺失率達91.1%，不補值
F02_flip的left_knee_x欄位，缺失率達86.7%，不補值
F02_flip的left_knee_y欄位，缺失率達86.7%，不補值
F03的left_wrist_x欄位，缺失率達61.1%，不補值
F03的left_wrist_y欄位，缺失率達61.1%，不補值
F03的right_wrist_x欄位，缺失率達94.4%，不補值
F03的right_wrist_y欄位，缺失率達94.4%，不補值
F03_flip的left_elbow_x欄位，缺失率達85.2%，不補值
F03_flip的left_elbow_y欄位，缺失率達85.2%，不補值
F03_flip的right_elbow_x欄位，缺失率達70.4%，不補值
F03_flip的right_elbow_y欄位，缺失率達70.4%，不補值
F03_fl

/tmp/ipython-input-43-1558798486.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  interpolated = (df.groupby('video').apply(lambda g: interpolate(g, threshold=3/5)).reset_index(drop=True))


## 每組依相同間隔取長度為9的序列

In [ ]:
def sequence(group):
  length = len(group)
  assert length % 9 == 0, f"長度必須為9的倍數,{length}"
  step = length // 9
  sequences = []
  for offset in range(step):
    indices = [i * step + offset for i in range(9)]
    seq = group.iloc[indices].copy()
    sequences.append(seq)
  return pd.concat(sequences, ignore_index=True)


# 先複製
seq = interpolated.copy()
seq['seq'] = (seq.groupby('video').cumcount()+1)

seq_9 = seq.groupby("video", group_keys=False).apply(sequence).reset_index(drop=True)
seq_9['label'] = 5

/tmp/ipython-input-47-3812101282.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  seq_9 = seq.groupby("video", group_keys=False).apply(sequence).reset_index(drop=True)


In [ ]:
seq_9.to_csv('/content/drive/MyDrive/訓練/data/seq_9_fall.csv', index=False)

# 資料預處理 - Dataset Video For Human Action Recognition

In [ ]:
csv_path_1 = "/content/drive/MyDrive/訓練/data/original_data/train_yolov7.csv"
df_1 = pd.read_csv(csv_path_1)

#只保留13個關鍵點
keypoints = [
  'video', 'label', 'frame',
  'nose_x', 'nose_y',
  'left_shoulder_x', 'left_shoulder_y',
  'right_shoulder_x', 'right_shoulder_y',
  'left_elbow_x', 'left_elbow_y',
  'right_elbow_x', 'right_elbow_y',
  'left_wrist_x', 'left_wrist_y',
  'right_wrist_x', 'right_wrist_y',
  'left_hip_x', 'left_hip_y',
  'right_hip_x', 'right_hip_y',
  'left_knee_x', 'left_knee_y',
  'right_knee_x', 'right_knee_y',
  'left_ankle_x', 'left_ankle_y',
  'right_ankle_x', 'right_ankle_y'
]
df_1 = df_1[keypoints]

# 刪除siting、standing、lying
df_1 = df_1[~df_1['label'].isin([1, 4, 6])]

df_1['group'] = df_1['video']

# 每列缺失值>7的，標記為無效列
df_1['missing_count'] = df_1.isna().sum(axis=1)
df_1['is_invalid'] = df_1['missing_count'] > 14

# 每個 group 判斷是否保留
def is_group_valid(group):
  label = group['label'].iloc[0]
  ratio = group['is_invalid'].sum() / len(group)
  if label == 5:
    return ratio <= 0.6
  else:
    return ratio <= 0.2 and not group['video'].iloc[0].endswith('flip.avi')

# 只保留有效group
valid_groups_1 = df_1.groupby('group').filter(is_group_valid)['group'].unique()
df_clean_1 = df_1[df_1['group'].isin(valid_groups_1)]

# df_clean_1.to_csv('/content/skeleton_interpolated.csv', index=False)

In [ ]:
csv_path_2 = "/content/drive/MyDrive/訓練/data/original_data/test_yolov7.csv"
df_2 = pd.read_csv(csv_path_2)

df_2 = df_2[df_2['label'] == 5]
df_2 = df_2[keypoints]

df_2['group'] = df_2['video']

df_2['missing_count'] = df_2.isna().sum(axis=1)
df_2['is_invalid'] = df_2['missing_count'] > 14

# group裡無效列>3/5的要刪除整組
group_valid_2 = (df_2.groupby('group')['is_invalid'].apply(lambda x: (x.sum() / len(x)) <= 0.6))

# 只保留有效group
valid_groups_2 = group_valid_2[group_valid_2].index
df_clean_2 = df_2[df_2['group'].isin(valid_groups_2)]

# df_clean_2.to_csv('/content/skeleton_interpolated.csv', index=False)

In [ ]:
df_all = pd.concat([df_clean_1, df_clean_2], ignore_index=True)

df_not_trimmed = []
df_trimmed = []

for _, group in df_all.groupby('video'):
  if group['is_invalid'].any():
    df_not_trimmed.append(group)
  else:
    length = len(group)
    remainder = length % 9
    if remainder > 0:
      df_trimmed.append(group.iloc[:-remainder].copy())
    else:
      df_not_trimmed.append(group)

df_all_ordered = pd.concat(df_not_trimmed + df_trimmed, ignore_index=True)

df_all_ordered = df_all_ordered.drop(columns=['missing_count', 'is_invalid', 'group'], errors='ignore')

df_all_ordered.to_csv('/content/drive/MyDrive/訓練/data/temp/URData.csv', index=False)


### 刪除一些需要線性補值較多的數據

In [ ]:
csv_path_3 = "/content/drive/MyDrive/訓練/data/temp/URData_clear.csv"
df_3 = pd.read_csv(csv_path_3)

def interpolate_trim(group, threshold=0.6):
  group_copy = group.copy()
  numeric_cols = group_copy.select_dtypes(include=['number']).columns

  for col in numeric_cols:
    missing_ratio = group_copy[col].isna().sum() / len(group_copy)
    if missing_ratio <= threshold:
      group_copy[col] = group_copy[col].interpolate(method='linear', limit_direction='both')
    else:
      print(f"{group_copy['video'].iloc[0]} 的 {col} 欄位缺失率達 {missing_ratio:.1%}，不補值")
  # 修剪
  length = len(group_copy)
  remainder = length % 9
  if remainder > 0:
    group_copy = group_copy.iloc[:-remainder]

  return group_copy


df_processed = df_3.groupby('video', group_keys=False).apply(interpolate_trim).reset_index(drop=True)

df_processed.to_csv("/content/skeleton_interpolated.csv", index=False)

/tmp/ipython-input-59-55986740.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_processed = df_3.groupby('video', group_keys=False).apply(interpolate_trim).reset_index(drop=True)


### 每組依相同間隔取長度為9的序列

In [ ]:
def sequence(group):
  length = len(group)
  assert length % 9 == 0, f"長度必須為9的倍數,{length}"
  step = length // 9
  sequences = []
  for offset in range(step):
    indices = [i * step + offset for i in range(9)]
    seq = group.iloc[indices].copy()
    sequences.append(seq)
  return pd.concat(sequences, ignore_index=True)

# 先複製
seq_pro = df_processed.copy()
seq_pro['seq'] = (seq_pro.groupby('video').cumcount()+1)

seq_pro_9 = seq_pro.groupby("video", group_keys=False).apply(sequence).reset_index(drop=True)

/tmp/ipython-input-60-3924739638.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  seq_pro_9 = seq_pro.groupby("video", group_keys=False).apply(sequence).reset_index(drop=True)


In [ ]:
seq_pro_9.to_csv('/content/drive/MyDrive/訓練/data/URData.csv', index=False)

## 合併

In [ ]:
cols_order = [
    'video', 'frame', 'seq',
    'nose_x', 'nose_y',
    'left_shoulder_x', 'left_shoulder_y',
    'right_shoulder_x', 'right_shoulder_y',
    'left_elbow_x', 'left_elbow_y',
    'right_elbow_x', 'right_elbow_y',
    'left_wrist_x', 'left_wrist_y',
    'right_wrist_x', 'right_wrist_y',
    'left_hip_x', 'left_hip_y',
    'right_hip_x', 'right_hip_y',
    'left_knee_x', 'left_knee_y',
    'right_knee_x', 'right_knee_y',
    'left_ankle_x', 'left_ankle_y',
    'right_ankle_x', 'right_ankle_y',
    'label'
]

seq_9 = seq_9[cols_order]

seq_pro_9 = seq_pro_9[cols_order]


combined_df = pd.concat([seq_9, seq_pro_9], ignore_index=True)

combined_df['seq'] = combined_df.groupby('video').cumcount() % 9 + 1

combined_df.to_csv('/content/drive/MyDrive/訓練/data/Data.csv', index=False)